In [1]:
## ETL nutcracker
# Conny Lin | June 6, 2020
# transform data from raw to ML ready data

In [2]:
pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'
csvname = 'trinity_cleaned_N2_etoh_b4tap.csv'
pMWTDB = '/Users/connylin/Dropbox/MWT/db/MWTDB.csv'

In [3]:
# import libraries
import os, sys, glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
# import local functions
sys.path.insert(1, '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/brainstation_capstone/0_lib')
import BrainStationLib as bs

In [4]:
# make legend
pchorelegend = os.path.join(pCapstone, 'legend_choreography.csv')
chorjavacall = 'tDfpemMwWlLaAkcsSbpdxyuvor1234'
chorelegend = bs.make_chor_output_legend(pchorelegend, chorjavacall)
pDropboxdb = '/Users/connylin/Dropbox/MWT/db'
pCobolt = '/Volumes/COBOLT'

In [5]:
len(chorjavacall)

30

Persistence data is duplicated. 

In [6]:
chorelegend['name']

0            time
1              id
2           frame
3     persistence
4            area
5         midline
6      morphwidth
7           width
8        relwidth
9          length
10      rellength
11         aspect
12      relaspect
13           kink
14          curve
15          speed
16        angular
17           bias
18    persistence
19            dir
20          loc_x
21          loc_y
22          vel_x
23          vel_y
24         orient
25           crab
26            tap
27           puff
28          stim3
29          stim4
Name: name, dtype: object

In [7]:
# preselect columns to load
column_index_keep = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,22,23,24,25]
column_names = chorelegend['name'][column_index_keep]

In [8]:
# TODO: look for data in db
# TODO: change pDropboxdb to pCobolt
mwtpaths_db = glob.glob(pDropboxdb+'/*/*/*/[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_[0-9][0-9][0-9][0-9][0-9][0-9]')
print(f'{len(mwtpaths_db)} mwt folders found')
# save this
df = pd.DataFrame({'mwtpath':mwtpaths_db})
df.to_csv(os.path.join(pCapstone, 'mwtpath.csv'))

7294 mwt folders found


In [9]:
def nutcracker_process_rawdata(pdata, mwtid):
    column_names_raw = ['time','id','frame','persistence','area','midline','morphwidth',
                        'width','relwidth','length','rellength','aspect','relaspect',
                        'kink','curve','speed','angular','bias','persistence','dir',
                        'loc_x','loc_y','vel_x','vel_y','orient','crab','tap','puff',
                        'stim3','stim4']  
    column_index_keep = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,22,23,24,25]
    # load data put in data frame
    df = pd.read_csv(pdata, delimiter=' ', header=None, usecols=column_index_keep, 
                     names=column_names, dtype=np.float64, engine='c')
    # remove data before 100s
    df.drop(axis=0, index=df.index[df['time']>100], inplace=True)
    # remove nan
    df.dropna(axis=0, inplace=True)
    # add mwtid column
    df.insert(0,'mwtid', np.tile(mwtid, df.shape[0]))
    # add etoh column
    if ('/N2_400mM/' in pdata):
        df.insert(0,'etoh', np.tile(1, df.shape[0]))
    else:
        df.insert(0,'etoh', np.tile(0, df.shape[0]))
    return df

In [10]:
def nutcracker_pick_consolidate_data(mwtpaths_db, pCobolt, pDropboxdb):
    # look for nutcracker files in this plate
    nutcracker_filelist = []
    for imwt, pmwt in enumerate(mwtpaths_db):
        pnutcracker = glob.glob(pmwt+'/*.nutcracker.*.dat')
        if len(pnutcracker) > 0:
            print(pmwt)
            # make storage for df
            df_store = []
            for ifile, pdata in enumerate(pnutcracker):
                print(f'\tprocessing {ifile}', end='\r')
                # get time data
                df = pd.read_csv(pdata, delimiter=' ', header=None, usecols=[0], 
                                 names=['time'], dtype=np.float64, engine='c')
                # see if data has time before 100s
                if sum(df['time']<100) > 0:
                    df = nutcracker_process_rawdata(pdata, imwt)
                    # add df to storage
                    df_store.append(df)
            # combine multiple nutcracker files (just before tap and only non NAN)
            df_mwt = pd.concat(df_store, ignore_index=True)
            print(f'\n\t{df_mwt.shape[0]} rows')
            # add etoh column

            # save csv in dropbox
            pmwt_dropbox = str.replace(pmwt, pCobolt, pDropboxdb)
            pdata_save_dropbox = os.path.join(pmwt_dropbox, 'nutcracker_100s.csv')
            nutcracker_filelist.append(pdata_save_dropbox)
            df_mwt.to_csv(pdata_save_dropbox, index=False)
            print(f'\tsaved nutcracker_100s.csv')
    return df_mwt, nutcracker_filelist

estimate data size: 33.1*1500/1000 = 50GB - more reasonable

In [11]:
df_mwt, nutcracker_filelist = nutcracker_pick_consolidate_data(mwtpaths_db, pCobolt, pDropboxdb)

/Users/connylin/Dropbox/MWT/db/MWT/20111114C_CL_100s30x10s10s/N2_400mM/20111114_141722
	processing 192
	66924 rows
	saved nutcracker_100s.csv
/Users/connylin/Dropbox/MWT/db/MWT/20111114C_CL_100s30x10s10s/N2/20111114_142532
	processing 295
	83976 rows
	saved nutcracker_100s.csv
/Users/connylin/Dropbox/MWT/db/MWT/20111114C_CL_100s30x10s10s/N2/20111114_140917
	processing 143
	89038 rows
	saved nutcracker_100s.csv


In [12]:
# estimate number of rows at the end
print('estimate of total rows without data reduction')
print(int((239938 / 3 * 1400) * (5*60/90) * (8000/1400)))
print('estimate of total rows with data reduction')
print(int((239938 / 3 * 1400)))


estimate of total rows without data reduction
2132782222
estimate of total rows with data reduction
111971066


In [13]:
# load and combine nutcracker_filelist
df_store = []
for filepath in nutcracker_filelist:
    df_store.append(pd.read_csv(filepath, dtype=np.float64, engine='c'))
data = pd.concat(df_store, ignore_index=True)
data


,etoh,mwtid,time,id,frame,persistence,area,midline,morphwidth,width,...,kink,curve,speed,angular,bias,dir,vel_x,vel_y,orient,crab
0,1.0,405.0,63.499,133.0,1575.0,67.5,0.231822,1.2165,0.1887,0.1512,...,6.7,19.7,0.2918,6.9,1.0,0.0,-0.1199,0.2660,114.8,0.0030
1,1.0,405.0,63.547,133.0,1576.0,67.5,0.229635,1.2372,0.1868,0.1512,...,10.3,19.7,0.2349,5.3,1.0,0.0,-0.0788,0.2213,115.1,0.0225
2,1.0,405.0,63.588,133.0,1577.0,67.5,0.232551,1.2302,0.1911,0.1512,...,15.9,20.5,0.2589,4.4,1.0,0.0,-0.1105,0.2342,115.3,0.0003
3,1.0,405.0,63.628,133.0,1578.0,67.5,0.234738,1.2213,0.1920,0.1485,...,16.6,17.3,0.2740,3.9,1.0,0.0,-0.1133,0.2495,115.4,0.0048
4,1.0,405.0,63.667,133.0,1579.0,67.5,0.231822,1.2273,0.1889,0.1485,...,17.5,17.2,0.2680,4.3,1.0,0.0,-0.0835,0.2547,115.6,0.0346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239933,0.0,411.0,94.345,150.0,2315.0,53.8,0.142155,1.4977,0.0649,0.2376,...,28.9,33.4,1.2151,40.5,1.0,0.0,-1.2098,-0.1129,10.6,0.1112
239934,0.0,411.0,94.391,150.0,2316.0,53.8,0.141426,1.5051,0.0663,0.2376,...,26.0,32.4,0.2333,1.7,1.0,0.0,-0.2295,-0.0421,10.5,0.0006
239935,0.0,411.0,94.433,150.0,2317.0,53.8,0.137781,1.5180,0.0660,0.2376,...,32.9,34.6,0.8067,39.4,1.0,0.0,0.8065,0.0184,10.5,0.1292
239936,0.0,411.0,94.473,150.0,2318.0,53.8,0.097686,0.9618,0.0672,0.1161,...,43.2,30.6,0.9227,39.7,0.0,0.0,0.9215,0.0464,14.5,0.1855


In [14]:
data.info()
size_file = int(sys.getsizeof(data)/1000**3/len(df_store)*1400)
print(f'estimate end size {size_file:.2f} GB')
del df_store

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239938 entries, 0 to 239937
Data columns (total 25 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   etoh         239938 non-null  float64
 1   mwtid        239938 non-null  float64
 2   time         239938 non-null  float64
 3   id           239938 non-null  float64
 4   frame        239938 non-null  float64
 5   persistence  239938 non-null  float64
 6   area         239938 non-null  float64
 7   midline      239938 non-null  float64
 8   morphwidth   239938 non-null  float64
 9   width        239938 non-null  float64
 10  relwidth     239938 non-null  float64
 11  length       239938 non-null  float64
 12  rellength    239938 non-null  float64
 13  aspect       239938 non-null  float64
 14  relaspect    239938 non-null  float64
 15  kink         239938 non-null  float64
 16  curve        239938 non-null  float64
 17  speed        239938 non-null  float64
 18  angular      239938 non-

In [15]:
# EDA
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
etoh,239938.0,0.278922,0.448470,0.00000,0.000000,0.000000,1.000000,1.000000
mwtid,239938.0,408.626487,2.409899,405.00000,405.000000,409.000000,411.000000,411.000000
time,239938.0,53.694809,26.645745,2.82300,30.731000,54.471000,76.521000,99.976000
id,239938.0,101.542428,132.901768,1.00000,17.000000,51.000000,122.000000,766.000000
frame,239938.0,1211.683022,613.676528,70.00000,710.000000,1182.000000,1673.000000,2476.000000
persistence,239938.0,130.935179,96.641314,14.10000,52.000000,103.600000,190.300000,390.300000
area,239938.0,0.164809,0.026620,0.03645,0.150174,0.166941,0.180063,0.290142
midline,239938.0,1.084889,0.105428,0.28120,1.027000,1.084100,1.147600,1.889400
morphwidth,239938.0,0.133476,0.022757,0.03590,0.121700,0.133300,0.145100,0.676100
width,239938.0,0.210135,0.072179,0.05400,0.153900,0.202500,0.253800,0.661500


In [17]:
# split X/y
# y column
y_column = ['etoh']
y = data[y_column].copy()
data.drop(columns=y_column, inplace=True)
y.to_csv(os.path.join(pCapstone, 'nutcracker_y.csv'), index=False)
# identifier column
identifier_column = ['id','mwtid']
data_identifiers = data[identifier_column].copy()
data.drop(columns=identifier_column, inplace=True)
data_identifiers.to_csv(os.path.join(pCapstone, 'nutcracker_identifier.csv'), index=False)
# save X
data.to_csv(os.path.join(pCapstone, 'nutcracker_X.csv'), index=False)